<a href="https://colab.research.google.com/github/ntshrocha/MAB605-2019.1/blob/master/Realimenta%C3%A7%C3%A3o_de_Relev%C3%A2ncia_Natasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Código dos exercícios anteriores

In [0]:
import re # expressões regulares
import functools # programação funcional
import numpy as np
import pandas as pd
import operator

# UTILS ########################################################################

def tokenize(string, separators, stopwords):
  regex = re.compile(f'[{"".join(separators)}]+')
  tokens = re.split(regex, string) # split on separators
  tokens = [x.lower() for x in tokens] # lowercase token
  tokens = filter(lambda x: x not in stopwords, tokens) # remove stopwords
  return list(filter(None, tokens)) # remove empty tokens

def incidence_matrix(tokenized_docs, alphabet = None):
  # Transforma array de arrays em um único array:
  flatten = lambda l: [item for sublist in l for item in sublist]

  if alphabet is None:
    # Filtra palavras únicas e retorna alfabeto com todas as palavras nos docs:
    alphabet = functools.reduce(lambda l, x: l if x in l else l+[x], flatten(tokenized_docs), [])

  IM = pd.DataFrame(columns=alphabet)
  # Para cada documento:
  for i, doc in enumerate(tokenized_docs):
    doc_name = f'{i+1}'
    # Cria linha para adicionar no dataframe
    row = pd.Series(np.zeros(len(alphabet)), index=alphabet, name=doc_name, dtype=int)
    # Para cada palavra do documento:
    for word in doc:
      row[word] += 1
    IM = IM.append(row)
  return IM

def get_tf(matrix):
  TF = matrix.copy().applymap(lambda f: 0 if f == 0 else 1 + np.log2(f))
  return TF

def get_idf(matrix):
  N = matrix.shape[0] # Número total de documentos
  IDF = matrix.astype(bool).sum() # Conta valores não nulos por coluna
  return IDF.apply(lambda n: np.log2(N/n))

def get_tf_idf(TF, IDF):
  TF = TF.where(TF != 0) # Zeros transformados em NaN para não atrapalhar contas
  w = TF.mul(IDF)
  return w.fillna(0) # Transforma NaN de volta em 0

# MODELO VETORIAL ##############################################################

def modelo_vetorial(docs, query):
  # Criação da matriz de incidência com frequência:
  IM_docs = incidence_matrix(docs)
  IM_query = incidence_matrix([query], alphabet = IM_docs.columns)
  
  # Frequência total de ocorrência dos termos:
  TF = get_tf(IM_docs)
  TF_query = get_tf(IM_query)
  
  # Frequência de documento:
  IDF = get_idf(IM_docs)
  
  # Ponderação TF-IDF:
  w = get_tf_idf(TF, IDF)
  w_query = get_tf_idf(TF_query, IDF)
  
  rank = {}
  
  def sim(w1, w2):
    return w1.dot(w2)/(np.linalg.norm(w1)*np.linalg.norm(w2))
  
  for name, vector in w.iterrows():
    rank[name] = sim(vector, w_query.T)[0]
  
  return sorted(rank.items(), key=lambda kv: -kv[1])

## Exercício 8 - Modelo de Rocchio

In [0]:
# DADOS DO PROBLEMA ############################################################

documentos = ['O peã e o caval são pec de xadrez. O caval é o melhor do jog.', 'A jog envolv a torr, o peã e o rei.', 'O peã lac o boi', 'Caval de rodei!', 'Polic o jog no xadrez.']

stopwords = ['a', 'o', 'e', 'é', 'de', 'do', 'no', 'são']

consulta = 'xadrez peã caval torr'

separadores = [' ',',','.','!','?']

R = [1, 2] # identificador dos documentos relevantes para a consulta q

alpha, beta, gama = 1, 0.75, 0.15 # parâmetros do método de Rocchio

In [0]:
# MODELO DE ROCCHIO ############################################################

def rocchio(docs, query, R, results, alpha = 1, beta = 0.75, gamma = 0.15):
  N = len(docs) # Número de documentos da coleção
  Dr = [] # Conjunto de documentos relevantes dentre os recuperados
  Dn = [] # Conjunto de documentos não relevantes dentre os recuperados
  
  # Criação da matriz de incidência com frequência:
  IM_docs = incidence_matrix(docs)
  IM_query = incidence_matrix([query], alphabet = IM_docs.columns)
  
  # Frequência total de ocorrência dos termos:
  TF = get_tf(IM_docs)
  TF_query = get_tf(IM_query)
  
  # Frequência de documento:
  IDF = get_idf(IM_docs)
  
  # Ponderação TF-IDF:
  W = get_tf_idf(TF, IDF)
  Q = get_tf_idf(TF_query, IDF)

  # Separa documentos do conjunto resultado em Dr e Dn
  for doc in results:
    n = doc[0]
    doc_vec = W.loc[n, :]
    Dr.append(doc_vec) if (int(n) in R) else Dn.append(doc_vec)
  
  Nr, Nn = len(Dr), len(Dn) # Número de documentos em Dr e Dn
  Dr = np.array(Dr)
  Dn = np.array(Dn)
  
  # Calcula vetor da consulta modificada
  Qm = alpha * Q + beta * np.sum(Dr, axis = 0)/Nr - gamma * np.sum(Dn, axis = 0)/Nn
  
  # Novo rankeamento com a consulta modificada:
  rank = {}
  
  def sim(w1, w2):
    return w1.dot(w2)/(np.linalg.norm(w1)*np.linalg.norm(w2))
  
  for name, vector in W.iterrows():
    rank[name] = sim(vector, Qm.T)[0]
  
  return Qm, sorted(rank.items(), key=lambda kv: -kv[1])

# EXEMPLO ############################################################

D = [tokenize(doc, separadores, stopwords) for doc in documentos]
Q = tokenize(consulta, separadores, stopwords)

resultado = modelo_vetorial(D, Q)
Qm, rank = rocchio(D, Q, R, resultado, alpha, beta, gama)

print("\033[1mRank do modelo vetorial:\033[0m")
print(resultado)
print("\033[1mNovo rank após 1 iteração:\033[0m")
print(rank)
print("\n\033[1mConsulta modificada:\033[0m")
Qm

Rank do modelo vetorial:
[('2', 0.4651729931620071), ('1', 0.415053375730601), ('4', 0.21298960013595078), ('5', 0.20532236528436032), ('3', 0.052555274134206874)]
Novo rank após 1 iteração:
[('2', 0.6371955792187908), ('1', 0.6217346178944333), ('4', 0.20973982246898742), ('5', 0.18159559038455), ('3', 0.02368245949389386)]

Consulta modificada:


,peã,caval,pec,xadrez,melhor,jog,envolv,torr,rei,lac,boi,rodei,polic
1,1.252842,2.247278,0.870723,1.751555,0.870723,0.515876,0.870723,3.192651,0.870723,-0.116096,-0.116096,-0.116096,-0.116096
